<a href="https://colab.research.google.com/github/pedrohcaminha/ProjetoPLN/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Baixando os dados

In [32]:
! pip install kaggle

In [33]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [34]:
! cp kaggle.json ~/.kaggle/

In [35]:
! chmod 600 ~/.kaggle/kaggle.json

In [36]:
! kaggle datasets download hassanamin/atis-airlinetravelinformationsystem

atis-airlinetravelinformationsystem.zip: Skipping, found more recently modified local copy (use --force to force download)


In [37]:
! unzip atis-airlinetravelinformationsystem.zip

Archive:  atis-airlinetravelinformationsystem.zip
replace atis_intents.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: atis_intents.csv        
  inflating: atis_intents_test.csv   
  inflating: atis_intents_train.csv  


In [38]:
! ls

atis-airlinetravelinformationsystem.zip  atis_intents_test.csv	 kaggle.json
atis_intents.csv			 atis_intents_train.csv  sample_data


# 2 Lendo e separando os dados

importando bibliotecas

In [59]:
import spacy
import csv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import spatial
import nltk
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers.embeddings import Embedding
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Input, Dense
from keras.models import Sequential

from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


dados de treino e teste

In [60]:
train = pd.read_csv("atis_intents_train.csv", header=None)
test = pd.read_csv("atis_intents_test.csv", header=None)

# 3 Pre-processamento

stop words

In [184]:
from nltk.corpus import stopwords
words = set(stopwords.words("english"))
print(words)

{"you'd", 'y', "hadn't", 'had', 'during', 'only', 'off', 'won', 'her', 'needn', 'through', 'a', 'after', "wasn't", 'these', 'itself', 'what', 't', "mightn't", 'haven', "don't", 'most', 'being', 'but', 'against', 's', 'out', 'shouldn', 'them', 're', 'can', 'their', 'didn', "you've", 'not', 'o', 'between', 'we', 'its', 'theirs', 'any', 'by', 'll', 'be', 'i', 'below', 'same', 'is', 'ma', 'been', 'did', 'has', 'he', 'that', 'our', 'an', 'd', 'hers', 'who', 'mightn', 'with', 'ourselves', 'or', 'own', 'the', 'than', 'doesn', 'wasn', 'and', 'hasn', "doesn't", "shan't", 'other', "wouldn't", 'this', 'ain', 'him', 'very', 'am', 'too', 'up', 'above', 'nor', 'down', 'you', 'shan', 've', 'for', 'have', 'over', 'will', 'themselves', 'do', 'about', 'on', 'now', 'when', 'yourself', 'where', 'does', 'into', 'having', 'she', 'herself', 'couldn', "mustn't", 'whom', 'some', 'which', "aren't", 'weren', 'further', "hasn't", 'why', 'here', 'those', 'aren', 'hadn', "isn't", 'are', "weren't", 'yourselves', 'my

In [185]:
train['text'] = train[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
test['text'] = test[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))

In [186]:
text = train['text']
labels = train[0]
test_text = test['text']
test_labels = test[0]

In [187]:
train.head()

,0,1,text
0,atis_flight,i want to fly from boston at 838 am and arriv...,want fly boston 838 arrive denver 1110 morning
1,atis_flight,what flights are available from pittsburgh to...,flights available pittsburgh baltimore thursda...
2,atis_flight_time,what is the arrival time in san francisco for...,arrival time san francisco 755 flight leaving ...
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfare tacoma orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fares pittsburgh philadelphia 1000 ...


# 4 vetorizando palavras

In [188]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(text)
word_index = tok.word_index

In [189]:
word_index

{'0900': 731,
 '1': 251,
 '10': 114,
 '100': 534,
 '1000': 188,
 '1020': 695,
 '1024': 533,
 '1026': 578,
 '1030': 554,
 '1039': 590,
 '1045': 648,
 '1055': 661,
 '1059': 625,
 '106': 562,
 '1083': 730,
 '11': 305,
 '110': 617,
 '1100': 570,
 '1110': 438,
 '1115': 366,
 '1130': 555,
 '1133': 735,
 '1145': 480,
 '1158': 633,
 '12': 138,
 '1200': 719,
 '1205': 670,
 '1209': 754,
 '1222': 474,
 '1230': 722,
 '124': 579,
 '1245': 367,
 '1288': 541,
 '1291': 411,
 '130': 640,
 '1300': 756,
 '139': 568,
 '150': 616,
 '1500': 752,
 '1505': 702,
 '1600': 602,
 '163': 772,
 '1700': 454,
 '1765': 458,
 '1850': 701,
 '19': 566,
 '1940': 763,
 '1991': 210,
 '1992': 369,
 '1993': 698,
 '2': 189,
 '200': 593,
 '201': 470,
 '21': 412,
 '210': 601,
 '2100': 377,
 '212': 739,
 '2134': 456,
 '2153': 557,
 '217': 488,
 '225': 632,
 '229': 727,
 '230': 356,
 '257': 542,
 '269': 509,
 '270': 431,
 '271': 626,
 '281': 321,
 '296': 423,
 '297': 595,
 '3': 222,
 '300': 688,
 '305': 740,
 '311': 682,
 '315': 5

In [190]:
max_vocab_size = len(word_index) + 1
input_length = 25

In [191]:
train_data_tokens = tok.texts_to_sequences(text)
test_data_tokens = tok.texts_to_sequences(test_text)

In [192]:
train_data_tokens

[[42, 22, 3, 328, 52, 6, 438, 19],
 [1, 36, 9, 11, 49, 19],
 [329, 145, 5, 7, 439, 2, 17, 16],
 [31, 229, 143, 93],
 [34, 29, 43, 9, 12, 188, 134],
 [32, 2, 101, 180, 139],
 [177, 102, 170, 2, 103, 10],
 [4, 1, 9, 105, 106, 49],
 [1, 3, 16],
 [177, 28, 30, 36, 6],
 [4, 1, 10, 5, 7],
 [4, 1, 5, 80, 69, 44, 90],
 [31, 2, 3, 246],
 [1, 11, 113, 20],
 [4, 24, 33, 43, 3, 6],
 [4, 28, 30, 6],
 [1, 6, 9, 17, 113, 20, 132, 20],
 [32, 57, 1, 55, 17, 11, 10, 10, 3, 3, 11],
 [18, 41, 1, 3, 9, 49, 82, 216],
 [21, 13, 22, 6, 9, 54, 15],
 [4, 1, 5, 80, 69],
 [18, 14, 24, 33, 1, 54, 6, 11],
 [386, 297, 170, 39, 15],
 [27, 13, 57, 123, 6, 9, 8],
 [27, 13, 148, 2, 8, 6],
 [83, 178, 6, 9, 8],
 [4, 1, 3, 9, 26, 82, 216, 46, 3, 189, 72, 20],
 [8, 28, 30],
 [217, 32, 159, 10, 3, 35, 67],
 [4, 31, 34, 29, 23, 11, 10],
 [4, 1, 45, 47, 59, 175, 15, 137, 440],
 [1, 45, 47, 62],
 [18, 60, 2, 68, 12, 5, 7, 41, 2, 527],
 [32, 2, 10, 5, 7],
 [43, 1, 9, 12],
 [4, 15, 24, 33, 1],
 [1, 58, 140, 193, 63, 64],
 [27, 13

In [193]:
train_input = pad_sequences(train_data_tokens, input_length)
test_input = pad_sequences(test_data_tokens, input_length)

In [194]:
train_input

array([[  0,   0,   0, ...,   6, 438,  19],
       [  0,   0,   0, ...,  11,  49,  19],
       [  0,   0,   0, ...,   2,  17,  16],
       ...,
       [  0,   0,   0, ...,  15,  22,   6],
       [  0,   0,   0, ...,   7,  78,   6],
       [  0,   0,   0, ...,   6,   5,   7]], dtype=int32)

# 5 codificando os rotulos

In [195]:
label_transformer = preprocessing.LabelEncoder()
label_transformer.fit(labels)

LabelEncoder()

In [196]:
print(labels)
labels = label_transformer.transform(labels)
print(labels)
test_labels = label_transformer.transform(test_labels)

0            atis_flight
1            atis_flight
2       atis_flight_time
3           atis_airfare
4           atis_airfare
              ...       
4829        atis_airfare
4830         atis_flight
4831        atis_airline
4832         atis_flight
4833         atis_flight
Name: 0, Length: 4834, dtype: object
[4 4 5 ... 3 4 4]


In [162]:
labels = to_categorical(np.asarray(labels))
test_labels = to_categorical(np.asarray(test_labels))

In [163]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 6 separação de treino e teste

In [164]:
X_train, X_val, y_train, y_val = train_test_split(train_input, labels, test_size=0.2, random_state=1)
X_train

array([[  0,   0,   0, ..., 214, 212,  33],
       [  0,   0,   0, ...,  42, 111,   6],
       [  0,   0,   0, ..., 184, 162,  70],
       ...,
       [  0,   0,   0, ...,   6,  12,  76],
       [  0,   0,   0, ...,  81,  75, 242],
       [  0,   0,   0, ...,  49, 174,  81]], dtype=int32)

# 7 deep learning

In [122]:
model = Sequential()
model.add(Embedding(max_vocab_size, 300, input_length=input_length, trainable=False))
model.add(Conv1D(filters=32, kernel_size=8, activation='selu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='selu'))
model.add(Dense(8, activation='sigmoid'))

In [123]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 300)           234000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 18, 32)            76832     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 32)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2890      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 88        
Total params: 313,810
Trainable params: 79,810
Non-trainable params: 234,000
___________________________________________

In [124]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, verbose=2)

Epoch 1/5
121/121 - 2s - loss: 0.9470 - accuracy: 0.7561
Epoch 2/5
121/121 - 1s - loss: 0.6262 - accuracy: 0.8244
Epoch 3/5
121/121 - 1s - loss: 0.4448 - accuracy: 0.8803
Epoch 4/5
121/121 - 1s - loss: 0.3279 - accuracy: 0.9105
Epoch 5/5
121/121 - 1s - loss: 0.2415 - accuracy: 0.9351


In [125]:
model.evaluate(X_val, y_val)

31/31 [==============================] - 0s 4ms/step - loss: 0.2931 - accuracy: 0.9214


[0.2931372821331024, 0.9214063882827759]

In [126]:
predictions = model.predict(test_input)

In [127]:
def acc(y_true, y_pred):
    return np.equal(np.argmax(y_true, axis=-1), np.argmax(y_pred, axis=-1)).mean()

print(acc(test_labels, predictions))

0.93625


In [169]:
def get_intent(sentence):
  data = [[sentence]]
  df = pd.DataFrame(data)
  entrada = df[0].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
  entrada = tok.texts_to_sequences(entrada)
  entrada = pad_sequences(entrada, input_length)
  prediction = model.predict(entrada)
  return np.argmin(prediction)

In [170]:
get_intent("Departure time of the flight to Seattle?")

6

In [175]:
label_transformer.inverse_transform(6)

ValueError: ignored